Introduction
=======

Ce tutoriel a pour but de vous illustrer différentes manières d'utiliser les services web (REST API) sur un serveur XiVO. Les exemples dans ce document utilisent le langage python, mais les principes démontrés peuvent être réutilisés dans d'autres langages de programmation.

Pour ce tutoriel nous allons nous concentrer sur l'utilisation de confd, le service de configuration des différentes resources sur un serveur xivo comme les utilisateurs, les extensions, les files d'attente, etc

Prérequis
=====

 * Un éditeur texte
 * Une version de python assez récente (2.7, 3.x)
 * La librairie python-requests
 
Sous debian, python-requests s'installe avec la commande suivante :

    apt-get install python-requests
    
Sinon, vous pouvez aussi installer avec pip, l'utilitaire de librarie pour python

    pip install requests

# Préparation

Avant d'envoyer des requêtes à confd, nous devons préparer quelques éléments de configuration :

 * Les données JSON
 * L'authentification
 
Pour éviter de recopier cette configuration à chaque requête, nous allons utiliser la notion de "session" offert par la librarie requests. La session permet de stocker un certain nombre de paramètres qui seront réutilisés à chaque fois que nous envoyons des requêtes.
 
L'authentification se fera grâce à un module externe disponible dans requests. **N'oubliez pas de configurer un utilisateur pour lesweb services dans la webi !**
 
Confd utilise le format JSON pour envoyer et recevoir des données. Nous allons ajouter ces headers HTTP pour signaler au serveur que nous allons envoyer et recevoir nos requêtes dans ce format.

In [3]:
import requests
import json
from pprint import pprint

requests.packages.urllib3.disable_warnings()

SERVER = "https://dev:9486/1.1"
USERNAME = "utilisateur"
PASSWORD = "motdepasse"


session = requests.Session()
session.verify = False
session.auth = requests.auth.HTTPDigestAuth(USERNAME, PASSWORD)
session.headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

# Lecture de données

Nous utilisons l'action GET en HTTP pour récupérer des donnés d'une ressource. Voici quelques exemples de requêtes pour récupérer des données à travers confd. Pour chaque exemple vous avez un exemple de code, suivi de la réponse.

## Liste

Par défaut, la requête retourne la liste de tous les éléments pour une ressource donnée sur un xivo. Dans l'exemple suivant nous listons tous les utilisateurs

In [4]:
url = "{}/users".format(SERVER)
response = session.get(url).json()

pprint(response)

{u'items': [{u'caller_id': u'"Mao Abdoulai"',
             u'description': u'',
             u'firstname': u'Mao',
             u'id': 3,
             u'language': None,
             u'lastname': u'Abdoulai',
             u'links': [{u'href': u'https://dev:9486/1.1/users/3',
                         u'rel': u'users'}],
             u'mobile_phone_number': None,
             u'music_on_hold': u'default',
             u'outgoing_caller_id': u'default',
             u'password': None,
             u'preprocess_subroutine': None,
             u'timezone': u'',
             u'userfield': u'',
             u'username': u''},
            {u'caller_id': u'"Mamasta Michel-Brunnemer"',
             u'description': u'',
             u'firstname': u'Mamasta',
             u'id': 2,
             u'language': None,
             u'lastname': u'Michel-Brunnemer',
             u'links': [{u'href': u'https://dev:9486/1.1/users/2',
                         u'rel': u'users'}],
             u'mobile_phone_

## Recherche

Vous pouvez faire une recherche avec un mot clé en ajoutant le paramètre "search=motcle" à la fin de l'URL. Dans l'exemple suivant nous utilisons l'option "params" de requests qui ajoutera automatiquent le paramètre à la fin de l'URL

In [5]:
url = "{}/users".format(SERVER)
query = {'search': 'sanderson'}
response = session.get(url, params=query).json()

pprint(response)

{u'items': [{u'caller_id': u'"Fod\xe9 Sanderson"',
             u'description': u'',
             u'firstname': u'Fod\xe9',
             u'id': 5,
             u'language': u'fr_FR',
             u'lastname': u'Sanderson',
             u'links': [{u'href': u'https://dev:9486/1.1/users/5',
                         u'rel': u'users'}],
             u'mobile_phone_number': None,
             u'music_on_hold': u'default',
             u'outgoing_caller_id': u'default',
             u'password': u'fode',
             u'preprocess_subroutine': None,
             u'timezone': u'America/Montreal',
             u'userfield': u'',
             u'username': u'fode'}],
 u'total': 1}


## Pagination

Vous pouvez limiter le nombre de résultats obtenus avec les paramètres suivants :

 * limit
     * nombre maximum d'éléments à retourner
 * skip
     * nombre d'éléments à passer outre dans la liste

In [6]:
url = "{}/users".format(SERVER)
query = {'limit': 2, 'skip': 1}
response = session.get(url, params=query).json()

pprint(response)

{u'items': [{u'caller_id': u'"Mamasta Michel-Brunnemer"',
             u'description': u'',
             u'firstname': u'Mamasta',
             u'id': 2,
             u'language': None,
             u'lastname': u'Michel-Brunnemer',
             u'links': [{u'href': u'https://dev:9486/1.1/users/2',
                         u'rel': u'users'}],
             u'mobile_phone_number': None,
             u'music_on_hold': u'default',
             u'outgoing_caller_id': u'default',
             u'password': u'mamasta',
             u'preprocess_subroutine': None,
             u'timezone': u'',
             u'userfield': u'',
             u'username': u'mamasta'},
            {u'caller_id': u'"Fod\xe9 Sanderson"',
             u'description': u'',
             u'firstname': u'Fod\xe9',
             u'id': 5,
             u'language': u'fr_FR',
             u'lastname': u'Sanderson',
             u'links': [{u'href': u'https://dev:9486/1.1/users/5',
                         u'rel': u'users'}],
 

## Triage

La liste peut se faire trier en ordre croissant (asc) ou décrossiant (desc) sur la colonne de votre choix. La colonne utilisé pour le tri se retrouve dans le paramètre "order"

In [7]:
url = "{}/users".format(SERVER)
query = {'order': 'lastname', 'direction': 'desc'}
response = session.get(url, params=query).json()

pprint(response)

{u'items': [{u'caller_id': u'"Fod\xe9 Sanderson"',
             u'description': u'',
             u'firstname': u'Fod\xe9',
             u'id': 5,
             u'language': u'fr_FR',
             u'lastname': u'Sanderson',
             u'links': [{u'href': u'https://dev:9486/1.1/users/5',
                         u'rel': u'users'}],
             u'mobile_phone_number': None,
             u'music_on_hold': u'default',
             u'outgoing_caller_id': u'default',
             u'password': u'fode',
             u'preprocess_subroutine': None,
             u'timezone': u'America/Montreal',
             u'userfield': u'',
             u'username': u'fode'},
            {u'caller_id': u'"Mamasta Michel-Brunnemer"',
             u'description': u'',
             u'firstname': u'Mamasta',
             u'id': 2,
             u'language': None,
             u'lastname': u'Michel-Brunnemer',
             u'links': [{u'href': u'https://dev:9486/1.1/users/2',
                         u'rel': u'u

## Récupérer une ressource

Pour récupérer un seul élément dans une liste, vous pouvez passer l'id de l'élément dans l'URL. Dans l'exemple suivant
nous récupérons les informations sur l'utilisateur avec l'id 2

In [8]:
user_id = 2
url = "{}/users/{}".format(SERVER, user_id)
response = session.get(url).json()

pprint(response)

{u'caller_id': u'"Mamasta Michel-Brunnemer"',
 u'description': u'',
 u'firstname': u'Mamasta',
 u'id': 2,
 u'language': None,
 u'lastname': u'Michel-Brunnemer',
 u'links': [{u'href': u'https://dev:9486/1.1/users/2', u'rel': u'users'}],
 u'mobile_phone_number': None,
 u'music_on_hold': u'default',
 u'outgoing_caller_id': u'default',
 u'password': u'mamasta',
 u'preprocess_subroutine': None,
 u'timezone': u'',
 u'userfield': u'',
 u'username': u'mamasta'}


# Écriture

L'écriture de données se fait avec l'action POST en HTTP. Comme mentionné précédemment, confd s'attend à recevoir des données sous format JSON. Heureusement, python nous offre une librairie qui permet de facilement transformer des données dans ce format.

## Création

Dans l'exemple suivant nous créons un utilisateur (John Doe), nous transformons les données en format JSON, puis nous envoyons ces données dans le body de la requête. Le serveur nous envoie une réponse confirmant la création de l'utilisateur

In [10]:
parameters = {'firstname': 'John',
              'lastname': 'Doe',
              'caller_id': '"Joe Cool"',
              'language': 'fr_FR',
              'username': 'joe',
              'password': 'password'}


url = "{}/users".format(SERVER)
encoded_parameters = json.dumps(parameters)

user = session.post(url, data=encoded_parameters).json()
pprint(user)

{u'caller_id': u'"Joe Cool"',
 u'description': None,
 u'firstname': u'John',
 u'id': 19,
 u'language': u'fr_FR',
 u'lastname': u'Doe',
 u'links': [{u'href': u'https://dev:9486/1.1/users/19', u'rel': u'users'}],
 u'mobile_phone_number': None,
 u'music_on_hold': None,
 u'outgoing_caller_id': None,
 u'password': u'password',
 u'preprocess_subroutine': None,
 u'timezone': None,
 u'userfield': None,
 u'username': u'joe'}


## Mise à jour de l'utilisateur

Lorsque nous mettons à jour une ressource, nous avons seulement besoin d'envoyer les champs qui ont été modifiés. Dans cet exemple, nous modifions seulement le prénom de l'utilisateur. Il est aussi important de spécifier l'id de l'utilisateur que nous voulons modifier dans l'URL

In [11]:
parameters = {'firstname': 'Johnny'}

url = "{}/users/{}".format(SERVER, user['id'])
encoded_parameters = json.dumps(parameters)

response = session.put(url, data=encoded_parameters)
pprint(response.status_code)

204


# Suppression

Pour supprimer une ressource, nous utilisons l'action DELETE en HTTP.

## Suppression d'un utilisateur

In [12]:
url = "{}/users/{}".format(SERVER, user['id'])

response = session.delete(url)
pprint(response.status_code)

204


# Exemples de manipulation

Voici quelques exemples un peu plus avancés de manipulation de données. Une bonne connaissance de la programmation en python vous sera bénéfique pour faire des manipulations encore plus poussées.

## Afficher la liste des noms d'utilisateurs complets

In [13]:
url = "{}/users".format(SERVER)
query = {'order': 'firstname', 'direction': 'desc', 'limit': 2}
response = session.get(url, params=query).json()

users = response['items']
for user in users:
    print "{} {}".format(user['firstname'], user['lastname'])

Mao Abdoulai
Mamasta Michel-Brunnemer


## Liste des adresses MAC pour les postes en mode autoprov

In [15]:
url = "{}/devices".format(SERVER)
response = session.get(url).json()

devices = response['items']
for device in devices:
    if device['status'] == 'autoprov':
        print device['mac']

00:08:5d:31:ef:e0


## Liste des extensions dans le contexte 'from-extern'

In [16]:
url = "{}/extensions".format(SERVER)
response = session.get(url).json()

extensions = response['items']
for extension in extensions:
    if extension['context'] == 'from-extern':
        print extension['exten']

1000
1001
2000
2100
2200
3000
4000


## Gestion d'erreurs

Pour des fins de démonstration, aucun exemple jusqu'à présent ne gère les erreurs sur la réception d'une réponse. En pratique, la gestion d'erreur est un élément important à considérer pour éviter de mauvaises surprises. Voici un exemple simplifié de comment aller vérifier si une réponse contient une erreur

In [17]:
url = "{}/users".format(SERVER)
response = session.post(url, data="{}")

if response.status_code >= 500:
    print response.text
elif response.status_code >= 400:
    errors = response.json()
    for error in errors:
        print error

Input Error - missing parameters: firstname


# Création d'utilisateur provisionné

Nous allons maintenant procédér à un exemple sur la création d'un utilisateur qui sera capable de recevoir et d'émettre des appels une fois terminé. Voici quelques notions à garder en tête :

 * User
     * Une personne utilisant l'IPBX
 * Extension
     * Le numéro composé pour rejoindre la personne
 * Line
     * Ce qui permet de relier ensemble un utilisateur et une extension
 * Device
     * Un poste, un téléphone physique.
   
   
Voici les étapes nécessaires à la création complete de l'utilisateur :

1. Création de l'utilisateur
2. Création de la ligne
3. Création de l'extension
4. Association de l'utilisateur et la ligne
5. Association de l'utilisateur et l'extension

Avec ces étapes nous avons un utilisateur capable d'émettre et de recevoir des appels à l'interne. Nous pouvons aussi permettre des appels à l'externe en ajoutant un appel entrant. Pour ce faire nous allons ajouter les étapes suivantes :

6. Création d'une extension pour l'appel entrant
7. Association de l'appel entrant et la ligne

## Création de l'utilisateur

<img src="users.png" />

In [18]:
parameters = {'firstname': 'Lord',
              'lastname': 'Sanderson'}


url = "{}/users".format(SERVER)
encoded_parameters = json.dumps(parameters)
user = session.post(url, data=encoded_parameters).json()

pprint(user)

{u'caller_id': None,
 u'description': None,
 u'firstname': u'Lord',
 u'id': 20,
 u'language': None,
 u'lastname': u'Sanderson',
 u'links': [{u'href': u'https://dev:9486/1.1/users/20', u'rel': u'users'}],
 u'mobile_phone_number': None,
 u'music_on_hold': None,
 u'outgoing_caller_id': None,
 u'password': None,
 u'preprocess_subroutine': None,
 u'timezone': None,
 u'userfield': None,
 u'username': None}


## Création de la ligne

<img src="lines.png" />

In [19]:
parameters = {'context': 'default',
              'device_slot': 1}

url = "{}/lines_sip".format(SERVER)
encoded_parameters = json.dumps(parameters)
line = session.post(url, data=encoded_parameters).json()

pprint(line)

{u'callerid': None,
 u'context': u'default',
 u'device_slot': 1,
 u'id': 15,
 u'links': [{u'href': u'https://dev:9486/1.1/lines_sip/15',
             u'rel': u'lines_sip'}],
 u'provisioning_extension': u'517423',
 u'secret': u'tdt0bgqm',
 u'username': u'okvvj4yz'}


## Création de l'extension

<img src="extensions.png" />

In [20]:
parameters = {'exten': '1100',
              'context': 'default'}

url = "{}/extensions".format(SERVER)
encoded_parameters = json.dumps(parameters)
extension = session.post(url, data=encoded_parameters).json()

pprint(extension)

{u'commented': False,
 u'context': u'default',
 u'exten': u'1100',
 u'id': 99,
 u'links': [{u'href': u'https://dev:9486/1.1/extensions/99',
             u'rel': u'extensions'}]}


## Association de l'utilisateur et la ligne

<img src="user_line.png" />

In [21]:
parameters = {'line_id': line['id']}

url = "{}/users/{}/lines".format(SERVER, user['id'])
encoded_parameters = json.dumps(parameters)
user_line = session.post(url, data=encoded_parameters).json()

pprint(user_line)

{u'line_id': 15,
 u'links': [{u'href': u'https://dev:9486/1.1/lines/15', u'rel': u'lines'},
            {u'href': u'https://dev:9486/1.1/users/20', u'rel': u'users'}],
 u'main_line': True,
 u'main_user': True,
 u'user_id': 20}


## Association de la ligne et de l'extension

 <img src="line_extension.png" />

In [22]:
parameters = {'extension_id': extension['id']}

url = "{}/lines/{}/extensions".format(SERVER, line['id'])
encoded_parameters = json.dumps(parameters)
line_extension = session.post(url, data=encoded_parameters).json()

pprint(line_extension)

{u'extension_id': 99,
 u'line_id': 15,
 u'links': [{u'href': u'https://dev:9486/1.1/extensions/99',
             u'rel': u'extensions'},
            {u'href': u'https://dev:9486/1.1/lines/15', u'rel': u'lines'}]}


## Création de l'extension pour l'appel entrant

In [24]:
parameters = {'exten': '2100',
              'context': 'from-extern'}

url = "{}/extensions".format(SERVER, line['id'])
encoded_parameters = json.dumps(parameters)
incall = session.post(url, data=encoded_parameters).json()

pprint(incall)

{u'commented': False,
 u'context': u'from-extern',
 u'exten': u'2100',
 u'id': 100,
 u'links': [{u'href': u'https://dev:9486/1.1/extensions/100',
             u'rel': u'extensions'}]}


## Association de l'extension pour l'appel entrant et la ligne

<img src="line_incall.png" />

In [25]:
parameters = {'extension_id': incall['id']}

url = "{}/lines/{}/extensions".format(SERVER, line['id'])
encoded_parameters = json.dumps(parameters)
line_incall = session.post(url, data=encoded_parameters).json()

pprint(line_incall)

{u'extension_id': 100,
 u'line_id': 15,
 u'links': [{u'href': u'https://dev:9486/1.1/extensions/100',
             u'rel': u'extensions'},
            {u'href': u'https://dev:9486/1.1/lines/15', u'rel': u'lines'}]}


## Exemple de toutes les étapes mis en commun

In [26]:
def create(url, parameters):
    full_url = SERVER + url
    encoded_parameters = json.dumps(parameters)
    
    response = session.post(full_url, data=encoded_parameters)
    
    if not response.status_code == requests.codes.created:
        raise Exception(response.text)
        
    return response.json()


parameters = {
    'user': {
        'firstname': 'Johnny',
        'lastname': 'Depp',
    },
    'line': {
        'context': 'default',
        'device_slot': 1,
    },
    'extension': {
        'exten': '1200',
        'context': 'default',
    },
    'incall': {
        'exten': '2200',
        'context': 'from-extern'
    }
}

user = create("/users", parameters['user'])
line = create("/lines_sip", parameters['line'])
extension = create("/extensions", parameters['extension'])
incall = create("/extensions", parameters['incall'])

associations = {
    'user_line': {'line_id': line['id']},
    'line_extension': {'extension_id': extension['id']},
    'line_incall': {'extension_id': incall['id']}
}

user_line = create("/users/{}/lines".format(user['id']), 
                   associations['user_line'])

line_extension = create("/lines/{}/extensions".format(line['id']), 
                        associations['line_extension'])

line_incall = create("/lines/{}/extensions".format(line['id']), 
                     associations['line_incall'])

print "provisioning extension: {}".format(line['provisioning_extension'])

provisioning extension: 113484


<img src="line_device.png" />